In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv("dataset.csv")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
data

,temperature,humidity,voltage,current,vibration,angle,status
0,25.1,60.0,1.0,0.1,0,0,0
1,25.2,60.2,1.1,0.2,0,0,0
2,25.3,60.4,1.2,0.3,0,0,0
3,25.4,60.6,1.3,0.4,0,0,0
4,25.5,60.8,1.4,0.5,0,0,0
...,...,...,...,...,...,...,...
995,49.5,104.5,12.7,0.6,1,1,1
996,49.6,104.6,12.8,0.7,1,1,1
997,49.7,104.7,12.9,0.8,1,1,1
998,49.8,104.8,13.0,0.9,1,1,1


Preprocessing

In [6]:
from sklearn.preprocessing import LabelEncoder
temp = LabelEncoder()
data['temperature'] = temp.fit_transform(data['temperature'])
hum = LabelEncoder()
data['humidity'] = hum.fit_transform(data['humidity'])
vol = LabelEncoder()
data['voltage'] = vol.fit_transform(data['voltage'])
cur = LabelEncoder()
data['current'] = cur.fit_transform(data['current'])

In [7]:
data

,temperature,humidity,voltage,current,vibration,angle,status
0,0,0,0,0,0,0,0
1,1,1,1,1,0,0,0
2,2,2,2,2,0,0,0
3,3,3,3,3,0,0,0
4,4,4,4,4,0,0,0
...,...,...,...,...,...,...,...
995,195,295,107,5,1,1,1
996,196,296,108,6,1,1,1
997,197,297,109,7,1,1,1
998,198,298,110,8,1,1,1


In [8]:
from keras.utils import np_utils
X = data.iloc[:,:-1]
y = data.iloc[:, -1]
y = np_utils.to_categorical(y)

train test split

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 0)

In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
import matplotlib.pyplot as plt
import seaborn as sns

Training model using network surgery

In [11]:
# import the necessary packages
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.engine import training

class FCHeadNet:
	@staticmethod
	def build(baseModel, classes, D):
		# initialize the head model that will be placed on top of
		# the base, then add a FC layer
		headModel = baseModel.output
		headModel = Flatten(name="flatten")(headModel)
		headModel = Dense(D, activation="relu")(headModel)
		headModel = Dropout(0.5)(headModel)

		# add a softmax layer
		headModel = Dense(classes, activation="softmax")(headModel)

		# return the model
		return headModel

In [12]:
import tensorflow as tf

def Conv_1D_Block(x, model_width, kernel):
    # 1D Convolutional Block with BatchNormalization
    x = tf.keras.layers.Conv1D(model_width, kernel, padding='same', kernel_initializer="he_normal")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    return x

class VGG:
    def __init__(self, length, num_channel, num_filters, problem_type='Regression', output_nums=1, dropout_rate=False):
        self.length = length
        self.num_channel = num_channel
        self.num_filters = num_filters
        self.problem_type = problem_type
        self.output_nums = output_nums
        self.dropout_rate = dropout_rate

    def VGG19(self):
        inputs = tf.keras.Input((self.length, self.num_channel))  # The input tensor
        # Block 1
        x = Conv_1D_Block(inputs, self.num_filters * (2 ** 0), 3)
        x = Conv_1D_Block(x, self.num_filters * (2 ** 0), 3)
        if x.shape[1] <= 2:
            x = tf.keras.layers.MaxPooling1D(pool_size=1, strides=2, padding="valid")(x)
        else:
            x = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding="valid")(x)

        # Block 2
        x = Conv_1D_Block(x, self.num_filters * (2 ** 1), 3)
        x = Conv_1D_Block(x, self.num_filters * (2 ** 1), 3)
        if x.shape[1] <= 2:
            x = tf.keras.layers.MaxPooling1D(pool_size=1, strides=2, padding="valid")(x)
        else:
            x = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding="valid")(x)

        # Block 3
        x = Conv_1D_Block(x, self.num_filters * (2 ** 2), 3)
        x = Conv_1D_Block(x, self.num_filters * (2 ** 2), 3)
        x = Conv_1D_Block(x, self.num_filters * (2 ** 2), 3)
        x = Conv_1D_Block(x, self.num_filters * (2 ** 2), 3)
        if x.shape[1] <= 2:
            x = tf.keras.layers.MaxPooling1D(pool_size=1, strides=2, padding="valid")(x)
        else:
            x = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding="valid")(x)

        # Block 4
        x = Conv_1D_Block(x, self.num_filters * (2 ** 3), 3)
        x = Conv_1D_Block(x, self.num_filters * (2 ** 3), 3)
        x = Conv_1D_Block(x, self.num_filters * (2 ** 3), 3)
        x = Conv_1D_Block(x, self.num_filters * (2 ** 3), 3)
        if x.shape[1] <= 2:
            x = tf.keras.layers.MaxPooling1D(pool_size=1, strides=2, padding="valid")(x)
        else:
            x = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding="valid")(x)

        # Block 5
        x = Conv_1D_Block(x, self.num_filters * (2 ** 3), 3)
        x = Conv_1D_Block(x, self.num_filters * (2 ** 3), 3)
        x = Conv_1D_Block(x, self.num_filters * (2 ** 3), 3)
        x = Conv_1D_Block(x, self.num_filters * (2 ** 3), 3)
        if x.shape[1] <= 2:
            x = tf.keras.layers.MaxPooling1D(pool_size=1, strides=2, padding="valid")(x)
        else:
            x = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding="valid")(x)

        # Fully Connected (MLP) block
        x = tf.keras.layers.Dense(4096, activation='relu')(x)
        x = tf.keras.layers.Dense(4096, activation='relu')(x)
        if self.dropout_rate:
            x = tf.keras.layers.Dropout(self.dropout_rate, name='Dropout')(x)
        outputs = tf.keras.layers.Dense(self.output_nums, activation='linear')(x)
        if self.problem_type == 'Classification':
            outputs = tf.keras.layers.Dense(self.output_nums, activation='softmax')(x)

        # Create model.
        model = training.Model(inputs, x, name='vgg19')

        return model

In [13]:
# load the VGG19 network, ensuring the head FC layer sets are left
# off
baseModel = VGG(length=6, num_channel=1, num_filters=16, problem_type="Classification", 
            output_nums=2, dropout_rate=0.5).VGG19()

# initialize the new head of the network, a set of FC layers
# followed by a softmax classifier
headModel = FCHeadNet.build(baseModel, 2, 256)

# place the head FC model on top of the base model -- this will
# become the actual model we will train
model = Model(inputs=baseModel.input, outputs=headModel)

In [14]:
# loop over all layers in the base model and freeze them so they
# will *not* be updated during the training process
for layer in baseModel.layers:
	layer.trainable = False

# compile our model (this needs to be done after our setting our
# layers to being non-trainable
print("[INFO] compiling model...")
opt = SGD(learning_rate=0.00001)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the head of the network for a few epochs (all other
# layers are frozen) -- this will allow the new FC layers to
# start to become initialized with actual "learned" values
# versus pure random
print("[INFO] training head...")
model.fit(X_train, y_train, batch_size=16,
	validation_data=(X_test, y_test), epochs=10,
	steps_per_epoch=len(X_train) // 16, verbose=1)

[INFO] compiling model...
[INFO] training head...
Epoch 1/10
50/50 [==============================] - 12s 32ms/step - loss: 0.6935 - accuracy: 0.5213 - val_loss: 0.6940 - val_accuracy: 0.5800
Epoch 2/10
50/50 [==============================] - 1s 12ms/step - loss: 0.6938 - accuracy: 0.5300 - val_loss: 0.6940 - val_accuracy: 0.5800
Epoch 3/10
50/50 [==============================] - 1s 13ms/step - loss: 0.6940 - accuracy: 0.4988 - val_loss: 0.6940 - val_accuracy: 0.5800
Epoch 4/10
50/50 [==============================] - 1s 12ms/step - loss: 0.6937 - accuracy: 0.5200 - val_loss: 0.6940 - val_accuracy: 0.5800
Epoch 5/10
50/50 [==============================] - 1s 12ms/step - loss: 0.6937 - accuracy: 0.5225 - val_loss: 0.6940 - val_accuracy: 0.5800
Epoch 6/10
50/50 [==============================] - 1s 12ms/step - loss: 0.6935 - accuracy: 0.5412 - val_loss: 0.6940 - val_accuracy: 0.5800
Epoch 7/10
50/50 [==============================] - 1s 12ms/step - loss: 0.6937 - accuracy: 0.5013 - va

In [15]:
# now that the head FC layers have been trained/initialized, lets
# unfreeze the final set of CONV layers and make them trainable
for layer in baseModel.layers[15:]:
	layer.trainable = True

# for the changes to the model to take affect we need to recompile
# the model, this time using SGD with a *very* small learning rate
print("[INFO] re-compiling model...")
opt = RMSprop(learning_rate=0.00001)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the model again, this time fine-tuning *both* the final set
# of CONV layers along with our set of FC layers
print("[INFO] fine-tuning model...")
model.fit(X_train, y_train, batch_size=16,
	validation_data=(X_test, y_test), epochs=10,
	steps_per_epoch=len(X_train) // 16, verbose=1)

[INFO] re-compiling model...
[INFO] fine-tuning model...
Epoch 1/10
50/50 [==============================] - 9s 53ms/step - loss: 0.6583 - accuracy: 0.6812 - val_loss: 0.6859 - val_accuracy: 0.5800
Epoch 2/10
50/50 [==============================] - 2s 33ms/step - loss: 0.5770 - accuracy: 0.7887 - val_loss: 0.6818 - val_accuracy: 0.5800
Epoch 3/10
50/50 [==============================] - 2s 34ms/step - loss: 0.5120 - accuracy: 0.8000 - val_loss: 0.6916 - val_accuracy: 0.5800
Epoch 4/10
50/50 [==============================] - 2s 33ms/step - loss: 0.4332 - accuracy: 0.8450 - val_loss: 0.7011 - val_accuracy: 0.5800
Epoch 5/10
50/50 [==============================] - 2s 33ms/step - loss: 0.3673 - accuracy: 0.8587 - val_loss: 0.6718 - val_accuracy: 0.6100
Epoch 6/10
50/50 [==============================] - 2s 33ms/step - loss: 0.3105 - accuracy: 0.8800 - val_loss: 0.5585 - val_accuracy: 0.7100
Epoch 7/10
50/50 [==============================] - 2s 34ms/step - loss: 0.2904 - accuracy: 0.895

In [16]:
# evaluate the network on the fine-tuned model

predictions = model.predict(X_test)
print("Classification Report: \n\n", classification_report(y_test.argmax(axis=1),
	predictions.argmax(axis=1)))



Classification Report: 

               precision    recall  f1-score   support

           0       0.87      1.00      0.93       116
           1       1.00      0.79      0.88        84

    accuracy                           0.91       200
   macro avg       0.93      0.89      0.90       200
weighted avg       0.92      0.91      0.91       200



### LSTM

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import LSTM
from keras.layers import Dropout

In [18]:
def lstm_model():
    regressor = Sequential()
    regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.25))
    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.25))
    regressor.add(Dense(units=9, activation='relu'))
    regressor.add(BatchNormalization())
    regressor.add(Dense(units = 2, activation='softmax'))
    return regressor

In [19]:
lstm = lstm_model()
print("[INFO] compiling...")
lstm.compile(loss="binary_crossentropy", optimizer=SGD(learning_rate=0.001),
	metrics=["accuracy"])

print("[INFO] training...")
lstm.fit(X_train, y_train, batch_size=16,
          validation_data=(X_test, y_test), epochs=20, verbose=1)

[INFO] compiling...
[INFO] training...
Epoch 1/20
50/50 [==============================] - 9s 48ms/step - loss: 0.6694 - accuracy: 0.6338 - val_loss: 0.6835 - val_accuracy: 0.5800
Epoch 2/20
50/50 [==============================] - 1s 12ms/step - loss: 0.5545 - accuracy: 0.8225 - val_loss: 0.6722 - val_accuracy: 0.8850
Epoch 3/20
50/50 [==============================] - 1s 11ms/step - loss: 0.4576 - accuracy: 0.8637 - val_loss: 0.6504 - val_accuracy: 0.8900
Epoch 4/20
50/50 [==============================] - 1s 12ms/step - loss: 0.3913 - accuracy: 0.8925 - val_loss: 0.6227 - val_accuracy: 0.9100
Epoch 5/20
50/50 [==============================] - 1s 16ms/step - loss: 0.3448 - accuracy: 0.9013 - val_loss: 0.5919 - val_accuracy: 0.9150
Epoch 6/20
50/50 [==============================] - 1s 17ms/step - loss: 0.3154 - accuracy: 0.9125 - val_loss: 0.5560 - val_accuracy: 0.9600
Epoch 7/20
50/50 [==============================] - 1s 17ms/step - loss: 0.3036 - accuracy: 0.9212 - val_loss: 0.51

In [20]:
# evaluate the model
predictions = lstm.predict(X_test)
print("Classification Report: \n", classification_report(y_test.argmax(axis=1),
	predictions.argmax(axis=1), target_names=["0", "1"]))
p=predictions.argmax(axis=1)


Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       116
           1       1.00      1.00      1.00        84

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



### proposed model

In [21]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import GlobalAveragePooling1D
from keras.layers import BatchNormalization
from keras.layers import concatenate
from keras.layers import Dense

In [22]:

def proposed_model():
    x = Input(shape=[6, 1]) # input
    y = Conv1D(kernel_size=1, filters=16, padding='same', activation='relu')(x)
    y = Conv1D(filters=24, kernel_size=1, padding='same', activation='relu')(y)
    y = BatchNormalization()(y)
    y = MaxPooling1D(pool_size=2)(y)
    y = Conv1D(filters=24, kernel_size=1, padding='same', activation='relu')(y)
    y = Conv1D(filters=24, kernel_size=1, padding='same', activation='relu')(y)
    y = BatchNormalization()(y)
    y = MaxPooling1D(pool_size=2)(y)
    y = Conv1D(filters=24, kernel_size=1, padding='same', activation='relu')(y)
    y = Conv1D(filters=16, kernel_size=1, padding='same', activation='relu')(y)
    y = GlobalAveragePooling1D()(y)
    y = Dense(units=81, activation='relu')(y)
    y = BatchNormalization()(y)
    y = Dense(units=27, activation='relu')(y)
    y = BatchNormalization()(y)
    y = Dense(2, activation='softmax')(y)

    model = Model(x, y)
    return model

In [23]:
model = proposed_model()
print("[INFO] compiling...")
model.compile(loss="categorical_crossentropy", optimizer=SGD(learning_rate=0.0001),
	metrics=["accuracy"])

print("[INFO] training...")
model.fit(X_train, y_train, batch_size=16, validation_data=(X_test, y_test), epochs=200, steps_per_epoch=len(X_train) // 16, verbose=1)

[INFO] compiling...
[INFO] training...
Epoch 1/200
50/50 [==============================] - 3s 17ms/step - loss: 0.8117 - accuracy: 0.5688 - val_loss: 0.7957 - val_accuracy: 0.4200
Epoch 2/200
50/50 [==============================] - 0s 9ms/step - loss: 0.4479 - accuracy: 0.8087 - val_loss: 0.8339 - val_accuracy: 0.4200
Epoch 3/200
50/50 [==============================] - 0s 9ms/step - loss: 0.4069 - accuracy: 0.8388 - val_loss: 0.8756 - val_accuracy: 0.4200
Epoch 4/200
50/50 [==============================] - 0s 9ms/step - loss: 0.3844 - accuracy: 0.8500 - val_loss: 0.8994 - val_accuracy: 0.4200
Epoch 5/200
50/50 [==============================] - 0s 9ms/step - loss: 0.3551 - accuracy: 0.8725 - val_loss: 0.8898 - val_accuracy: 0.4200
Epoch 6/200
50/50 [==============================] - 0s 9ms/step - loss: 0.3497 - accuracy: 0.8637 - val_loss: 0.8454 - val_accuracy: 0.4200
Epoch 7/200
50/50 [==============================] - 0s 9ms/step - loss: 0.3044 - accuracy: 0.9075 - val_loss: 0.7

In [24]:
# evaluate the network on the fine-tuned model
predictions = model.predict(X_test)
print("Classification Report: \n\n", classification_report(y_test.argmax(axis=1),
	predictions.argmax(axis=1), target_names=["0", "1"]))
p=predictions.argmax(axis=1)


Classification Report: 

               precision    recall  f1-score   support

           0       0.98      1.00      0.99       116
           1       1.00      0.98      0.99        84

    accuracy                           0.99       200
   macro avg       0.99      0.99      0.99       200
weighted avg       0.99      0.99      0.99       200



In [25]:
model.save("model.h5")

In [26]:
from keras.models import load_model
model = load_model("model.h5")

### Prediction

In [27]:
!pip install paho.mqtt

     |████████████████████████████████| 99 kB 4.1 MB/s 
  Created wheel for paho.mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62133 sha256=6c323ae3a2a5e5415929bbcf38b0394783309e839a204d9eef2504f5c90fa2e0
  Stored in directory: /root/.cache/pip/wheels/d0/bf/ac/2b3f43f8c6fcd0f4ba5395397458c521eb0b52d33b574a5a40
Successfully built paho.mqtt


In [ ]:
import paho.mqtt.client as mqtt
import requests
import pickle
import json
import time
import warnings
warnings.filterwarnings("ignore")

def on_message(client, userdata, message):
    print("li")
    data1 =[]
    receivedstring = str(message.payload.decode("utf-8"))
    data1=receivedstring.split(",")
    model = pickle.load(open("model.h5", 'rb'))
    data = []
    data.append(float(data1[0]))
    data.append(float(data1[1]))
    data.append(float(data1[2]))
    data.append(float(data1[3]))
    data.append(int(data1[4]))
    data.append(int(data1[5]))
    pred = model.predict([data])
    print(data)
    status = pred[0]
    print(status)
    vib = data[-2]
    angle = data[-1]
    client.publish("equip_failure_input", str(status))
    if status == 1:
        if vib == 1:
            client.publish("vels545", str(status)+","+str("MOTOR STOPPED DUE TO VIBRATION"))
        if angle == 1:
            client.publish("vels545", str(status)+","+str("MOTOR STOPPED DUE TO ANGLE CHANGE"))
    else:
        client.publish("vels545", str(status)+","+str("MOTOR RUNNING"))

broker_address="broker.hivemq.com"
client = mqtt.Client("SIEORA") 
client.connect(broker_address) 
client.on_message=on_message 
client.subscribe("equip_failure_output")

while True:
   
    client.loop_start()
    